In [1]:
import numpy as np 
import keras 
import tensorflow as tf 
from keras import layers,Model,Sequential,activations
from keras.callbacks import ModelCheckpoint, LearningRateScheduler 
from keras.callbacks import ReduceLROnPlateau 
from keras.regularizers import l2 
from keras.optimizers import Adam 
import matplotlib.pyplot as plt 
import keras.backend.tensorflow_backend as KTF 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE' 
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
# config.gpu_options.allow_growth=True #不全部占满显存, 按需分配
sess = tf.Session(config=config)

Using TensorFlow backend.
/home/liyanni/anaconda3/envs/adv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/liyanni/anaconda3/envs/adv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/liyanni/anaconda3/envs/adv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/liyanni/anaconda3/en

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train_mean = np.mean(x_train, axis=0)
x_test -= x_train_mean
x_train -= x_train_mean
model = keras.models.load_model("../defence/models/cifar10/cifar10_ResNet56v2.h5")
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# from scipy.io import loadmat
# test = loadmat("../test_32x32.mat")
# train = loadmat("../train_32x32.mat")
# def reformat(samples, labels):
#     samples = np.transpose(samples, (3, 0, 1, 2))
#     labels = keras.utils.to_categorical(labels-1, 10)
#     return samples, labels
# x_train, y_train = reformat(train['X'], train['y'])
# x_test, y_test = reformat(test['X'], test['y'])
# x_train, x_test = x_train / 255.0, x_test / 255.0
# x_train_mean = np.mean(x_train, axis=0)
# x_test -= x_train_mean
# x_train -= x_train_mean
# model = keras.models.load_model("../models/svhn/svhn_ResNet56v2.h5")

# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0
# y_train = keras.utils.to_categorical(y_train, 10)
# y_test = keras.utils.to_categorical(y_test, 10)
# x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
# x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
# model = keras.models.load_model("../models/mnist/mnist_model.008.h5")

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- [Errno -2] Name or service not known

In [23]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 3s 313us/step


[0.030074486126316333, 0.9916]

In [39]:
# logits = Model(inputs = model.input,
#                outputs = model.get_layer('flatten_1').output)
logits = Model(inputs = model.input,
               outputs = model.get_layer('dense_1').output)

In [40]:
logits.trainable = False
logits.name = 'cnn_logits'
fx_train = logits.predict(x_train)
fx_test = logits.predict(x_test)

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [41]:
# classifier
# 拿到model的最后一层
# classifier = Sequential()
# classifier.add(model.layers[-1])
# classifier.build((None,256))
# classifier.compile(loss='mean_absolute_error',
#                    optimizer=Adam(),
#                    metrics=['accuracy']
#                 )
# classifier.trainable=False

classifier = Sequential()
classifier.add(model.layers[-1])
classifier.build((None,128))
classifier.compile(loss='mean_absolute_error',
                   optimizer=Adam(),
                   metrics=['accuracy']
                )
classifier.trainable=False

In [55]:
# #  cifar10 数据集LGD 模型实现
# def ID():
#     x = layers.Input(shape=(32, 32, 3))
#     # forword
#     x1 = C2(x,64) # (32,32,64)
#     x2 = forword_C3(x1,128) # (16,16,128)
#     x3 = forword_C3(x2,256) # (8,8,256)
#     x4 = forword_C3(x3,256) # (4,4,256)
    
#     # backword    
#     x3 = F(x3,x4) # (8,8,512)
#     x3 = backword_C3(x3,256) #(8,8,256)
#     x2 = F(x2,x3) #(16,16,384)
#     x2 = backword_C3(x2,128) #(16,16,128)
#     x1 = F(x1,x2) #(32,32,192)
#     x1 = C2(x1,64) #(32,32,64)
    
#     # x + noise    
#     noise = layers.Conv2D(filters=3,kernel_size=(1,1))(x1) #(32,32,3)
#     y = layers.Add()([x,noise])
    
#     # get logits
#     y = logits(y)
#     return Model(inputs = x,outputs = y)
    
# def C2(x,out_channels):
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     return x
    
# def forword_C3(x,out_channels):
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     return x

# def backword_C3(x,out_channels):
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
#     return x

# def F(input1,input2):
#     input2 = layers.UpSampling2D(interpolation='bilinear')(input2)
#     output = layers.concatenate([input1,input2],axis=-1)
#     return output


In [ ]:
#  cifar10 数据集LGD 模型实现
def ID():
    x = layers.Input(shape=(28, 28, 1))
    # forword
    x1 = C2(x,32) # (28,28,64)
    x2 = forword_C3(x1,64) # (14,14,64)
    x3 = forword_C3(x2,128) # (7,7,128)
    
    # backword    
    x2 = F(x2,x3) #(14,14,192)
    x2 = backword_C3(x2,64) #(14,14,64)
    x1 = F(x1,x2) #(28,28,128)
    x1 = C2(x1,32) #(28,28,32)
    
    # x + noise    
    noise = layers.Conv2D(filters=1,kernel_size=(1,1))(x1) #(28,28,1)
    y = layers.Add()([x,noise])
    
    # get logits
    y = logits(y)
    return Model(inputs = x,outputs = y)
    
def C2(x,out_channels):
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x
    
def forword_C3(x,out_channels):
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def backword_C3(x,out_channels):
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=out_channels,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def F(input1,input2):
    input2 = layers.UpSampling2D(interpolation='bilinear')(input2)
    output = layers.concatenate([input1,input2],axis=-1)
    return output


In [44]:
# 定义回调函数
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 0.001
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('lr:',lr)
    return lr
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'svhn_LGD_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]
# callbacks1 = [lr_reducer, lr_scheduler,checkpoint]

In [98]:
x_adv = np.load("../adv_data/mnist/bim/0.25x_adv.npy")
# label = np.append(y_train,y_test,axis=0)

In [99]:
x_train_adv = x_adv[:60000]
x_test_adv = x_adv[60000:]

In [100]:
model.evaluate(x_test_adv,y_test)

10000/10000 [==============================] - 1s 137us/step


[2.369888822746277, 0.5229]

In [101]:
classifier.evaluate(fx_test,y_test)

10000/10000 [==============================] - 1s 87us/step


[0.002985491929484033, 0.9884]

In [102]:
denoiser = ID()
denoiser.compile(loss='mean_absolute_error',
              optimizer=Adam(lr = lr_schedule(0))
                )
              
batch_size = 128
epochs = 20
denoiser.fit(x_train_adv,fx_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test_adv,fx_test),
              shuffle=True,
#               callbacks=callbacks1
            )


lr: 0.001
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 58s 962us/step - loss: 1.4941 - val_loss: 0.4373
Epoch 2/20
60000/60000 [==============================] - 40s 673us/step - loss: 0.2577 - val_loss: 0.2025
Epoch 3/20
60000/60000 [==============================] - 40s 672us/step - loss: 0.1617 - val_loss: 0.1480
Epoch 4/20
60000/60000 [==============================] - 40s 674us/step - loss: 0.1364 - val_loss: 0.0936
Epoch 5/20
60000/60000 [==============================] - 40s 675us/step - loss: 0.1280 - val_loss: 0.1081
Epoch 6/20
60000/60000 [==============================] - 40s 675us/step - loss: 0.1236 - val_loss: 0.0987
Epoch 7/20
27008/60000 [============>.................] - ETA: 21s - loss: 0.1219

KeyboardInterrupt: 

In [104]:
tmp = iD.predict(x_test)
classifier.evaluate(tmp,y_test)

10000/10000 [==============================] - 1s 86us/step


[0.0025707527576449138, 0.9903]

In [105]:
fx_train_adv = iD.predict(x_train_adv)
fx_test_adv = iD.predict(x_test_adv)
fx_train = iD.predict(x_train)
fx_test = iD.predict(x_test)

In [106]:
train_group = []
(_, train_label), (_,_) = keras.datasets.cifar10.load_data()
# train_label = loadmat("../train_32x32.mat")["y"]-1
# (_, train_label), (_,_) = keras.datasets.mnist.load_data()
train_label = train_label.reshape(-1)
for i in range(10):
    train_index = np.argwhere(train_label == i).reshape(-1)
    tmp = np.append(fx_train[train_index],fx_train_adv[train_index],axis=0)
    train_group.append(tmp)
# shuffle
for i in range(10):
    permutation = np.random.permutation(train_group[i].shape[0])
    train_group[i] = train_group[i][permutation]

In [107]:
classifier.evaluate(fx_test_adv,y_test)

10000/10000 [==============================] - 1s 84us/step


[0.0031606624952421046, 0.9886]

In [ ]:
classifier.evaluate(fx_test,y_test)

 6240/10000 [=================>............] - ETA: 0s

In [71]:
# def Denoiser5():
#     x = layers.Input(shape=(256,))
    
#     x1 = layers.Dense(256)(x)
#     layers.Dropout(0.2)
#     x1 = layers.BatchNormalization()(x1)
#     x1 = layers.ReLU()(x1)
    
#     x2 = layers.Dense(128)(x1)
#     layers.Dropout(0.2)
#     x2 = layers.BatchNormalization()(x2)
#     x2 = layers.ReLU()(x2)
    
    
#     x3 = layers.Dense(64)(x2)
#     layers.Dropout(0.2)
#     x3 = layers.BatchNormalization()(x3)
#     x3 = layers.ReLU()(x3)
    
#     x4 = layers.Dense(32)(x3)
#     layers.Dropout(0.2)
#     x4 = layers.BatchNormalization()(x4)
#     x4 = layers.ReLU()(x4)
     
#     x4 = layers.Dense(64)(x4)
#     layers.Dropout(0.2)
#     x4 = layers.BatchNormalization()(x4)
#     x4 = layers.ReLU()(x4)
    
#     x4 = layers.add([x3,x4])
    
#     x4 = layers.Dense(128)(x4)
#     layers.Dropout(0.2)

#     x4 = layers.BatchNormalization()(x4)
#     x4 = layers.ReLU()(x4)
    
#     x4 = layers.add([x2,x4])
    
#     x4 = layers.Dense(256)(x4)
#     layers.Dropout(0.2)
#     x4 = layers.BatchNormalization()(x4)
#     x4 = layers.ReLU()(x4)
    
#     x4 = layers.add([x1,x4])
    
# #     x4 = layers.add([x,x4])
#     x4 = classifier(x4)
#     return Model(inputs = x,outputs = x4)

In [76]:
def HR():
    x = layers.Input(shape=(128,))
    
    x2 = layers.Dense(128)(x)
    layers.Dropout(0.2)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.ReLU()(x2)
    
    
    x3 = layers.Dense(64)(x2)
    layers.Dropout(0.2)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.ReLU()(x3)
    
    x4 = layers.Dense(32)(x3)
    layers.Dropout(0.2)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.ReLU()(x4)
     
    x4 = layers.Dense(64)(x4)
    layers.Dropout(0.2)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.ReLU()(x4)
    
    x4 = layers.add([x3,x4])
    
    x4 = layers.Dense(128)(x4)
    layers.Dropout(0.2)

    x4 = layers.BatchNormalization()(x4)
    x4 = layers.ReLU()(x4)
    
    x4 = layers.add([x2,x4])
    
#     x4 = layers.add([x,x4])
    x4 = classifier(x4)
    return Model(inputs = x,outputs = x4)

In [72]:
def generator(batch_size):
    while True:
        x,y=[],[]
        for _ in range(96):
            label = np.random.randint(10)
            length = len(train_group[label])
            v1=np.random.randint(length)
            v2=np.random.randint(length)
            rate1 = np.random.randint(1,11)/10
            rate2 = 1-rate1
            x.append(rate1*train_group[label][v1]+rate2*train_group[label][v2])
            lb = np.zeros(10)
            lb[label]=1
            y.append(lb)
#         for _ in range(32):
#             label1 = np.random.randint(10)
#             label2 = np.random.randint(10)
#             length1 = len(train_group[label1])
#             length2 = len(train_group[label2])
#             v1=np.random.randint(length1)
#             v2=np.random.randint(length2)
#             rate1 = np.random.randint(1,11)/10
#             rate2 = 1-rate1
#             x.append(rate1*train_group[label1][v1]+rate2*train_group[label2][v2])
#             lb1 = np.zeros(10)
#             lb1[label1]=rate1
#             lb2 = np.zeros(10)
#             lb2[label2]=rate2
#             y.append(lb1+lb2)
        x = np.array(x)
        y = np.array(y)
        yield x,y
dataGen = generator(128)

In [ ]:
hr = HR()
# denoiser5.layers[-1].trainable = False
hr.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr = lr_schedule(0)),
                  metrics=['accuracy']
                )
hr.fit_generator(dataGen,
              steps_per_epoch=500,
              epochs=80, 
              verbose=1, 
              validation_data=(fx_test_adv,y_test),
#               validation_data=valDataGen,
#               validation_steps=10000//128,
              callbacks=callbacks)

In [83]:
hr.evaluate(fx_test_adv,y_test)

26032/26032 [==============================] - 6s 237us/step


[0.16063544119879392, 0.9617393976644131]